This kernel relates to https://www.kaggle.com/c/house-prices-advanced-regression-techniques

All my work related to this challenge at
https://github.com/mulargui/kaggle-house-prices-advanced-regression-techniques

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt


In [15]:
####### DATA PREPARATION #####
#load data
dftrain=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv', index_col="Id")
dftest=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv', index_col="Id")

#split train data in features and labels
y = dftrain.SalePrice
x = dftrain.drop('SalePrice', axis=1)

# split test data in features and Ids
Ids = dftest.index
x_predict = dftest

# one data set with all features
X = pd.concat([x,x_predict],keys=[0,1])

In [16]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
#print(X.dtypes)
#print(X.isnull().sum(axis = 0))
#print(X['3SsnPorch'].head())
#print(X.3SsnPorch.head())
print(X[X['MSZoning'].isnull()])
#print(X.MoSold.unique())
#print(X[X['PoolArea'] == 0])
#print(X['YearBuilt'].max())
#X['Residential'] = np.where(X.MSZoning.isin(['FV']), 1, 0)
#X['tmp'] = np.where((X.Condition1.isin(['RRNn','RRNe','RRan','RRae'])) | (X.Condition2.isin(['RRNn','RRNe','RRan','RRae'])), 1, 0)
#print(X[X['tmp'] == 1])
#print(X[X.MSZoning.isin(['RH'])])


        MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
  Id                                                                      
1 1916          30      NaN        109.0    21780   Grvl   NaN      Reg   
  2217          20      NaN         80.0    14584   Pave   NaN      Reg   
  2251          70      NaN          NaN    56600   Pave   NaN      IR1   
  2905          20      NaN        125.0    31250   Pave   NaN      Reg   

       LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
  Id                                                                       
1 1916         Lvl       NaN    Inside       Gtl       IDOTRR       Norm   
  2217         Low    AllPub    Inside       Mod       IDOTRR       Norm   
  2251         Low    AllPub    Inside       Gtl       IDOTRR       Norm   
  2905         Lvl    AllPub    Inside       Gtl      Mitchel     Artery   

       Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
  Id          

In [14]:
###### FEATURE ENGINEERING #####

#to tranform cathegorical to values
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()

#nothing to do
#LotArea
#OverallQual
#OverallCond
#YearBuilt
#YearRemodAdd
#FullBath
#HalfBath
#YrSold
#WoodDeckSF
#OpenPorchSF
#EnclosedPorch
#3SsnPorch
#ScreenPorch
#1stFlrSF
#2ndFlrSF
#LowQualFinSF
#GrLivArea
#BedroomAbvGr
#KitchenAbvGr
#TotRmsAbvGrd
#Fireplaces
#MiscVal
#MoSold

#NEW FEATURES
#residential area
X['Residential'] = np.where(X.MSZoning.isin(['FV','RH','RL','RP','RM']), 1, 0)
#near railroad
X['Railroad'] = np.where((X.Condition1.isin(['RRNn','RRNe','RRan','RRae'])) | (X.Condition2.isin(['RRNn','RRNe','RRan','RRae'])), 1, 0)

#zone features
X["Neighborhood"] = X.Neighborhood.fillna("NA")
X["Neighborhood"] = lb_make.fit_transform(X["Neighborhood"])
X["MSZoning"] = X.MSZoning.fillna("NA")
X["MSZoning"] = lb_make.fit_transform(X["MSZoning"])
X["Condition1"] = lb_make.fit_transform(X["Condition1"])
X["Condition2"] = lb_make.fit_transform(X["Condition2"])

#geo features
X.LotShape.replace({"IR3":1,"IR2":2,"IR1":3,"Reg":4},inplace=True)
#very opinionated scoring
X.LandContour.replace({"Bnk":1,"HLS":2,"Low":3,"Lvl":4},inplace=True)
X["LotConfig"] = lb_make.fit_transform(X["LotConfig"])
X["LandSlope"] = lb_make.fit_transform(X["LandSlope"])

#Exterior features
X.ExterQual.replace({"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5},inplace=True)
X.ExterCond.replace({"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5},inplace=True)
X.Street.replace({"Grvl":1, "Pave":2},inplace=True)
#NaNs are NAs
X["Alley"] = X.Alley.fillna("NA")
X.Alley.replace({"NA":0, "Grvl":1, "Pave":2},inplace=True)
X["LotFrontage"] = X.LotFrontage.fillna(X.LotFrontage.mean())
X["MSSubClass"] = lb_make.fit_transform(X["MSSubClass"])
X["BldgType"] = lb_make.fit_transform(X["BldgType"])
X["HouseStyle"] = lb_make.fit_transform(X["HouseStyle"])
X["RoofStyle"] = lb_make.fit_transform(X["RoofStyle"])
X["RoofMatl"] = lb_make.fit_transform(X["RoofMatl"])
X["Foundation"] = lb_make.fit_transform(X["Foundation"])
X["Exterior1st"] = X.Exterior1st.fillna("NA")
X["Exterior1st"] = lb_make.fit_transform(X["Exterior1st"])
X["Exterior2nd"] = X.Exterior2nd.fillna("NA")
X["Exterior2nd"] = lb_make.fit_transform(X["Exterior2nd"])

#Masonry features
#all those that have no Masonry veneer area have no Masonry veneer type
X["MasVnrArea"] = X.MasVnrArea.fillna(0)
X["MasVnrType"] = X.MasVnrType.fillna("None")
X["MasVnrType"] = lb_make.fit_transform(X["MasVnrType"])
#row 2611 has area(198.0) but NaN type

#Interior
X.KitchenQual.replace({"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5},inplace=True)
#only one NaN
X["KitchenQual"] = X.KitchenQual.fillna(X.KitchenQual.mean())
#NaNs are NA
X["FireplaceQu"] = X.FireplaceQu.fillna("NA")
X.FireplaceQu.replace({"NA":0, "Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5},inplace=True)

#Basement features
#row 2121 has has all basement area features NaN
X["BsmtFinSF1"] = X.BsmtFinSF1.fillna(0)
X["BsmtFinSF2"] = X.BsmtFinSF2.fillna(0)
X["BsmtUnfSF"] = X.BsmtUnfSF.fillna(0)
X["TotalBsmtSF"] = X.TotalBsmtSF.fillna(0)
#NaNs are NAs
X["BsmtFinType1"] = X.BsmtFinType1.fillna("NA")
X["BsmtFinType2"] = X.BsmtFinType2.fillna("NA")
X["BsmtQual"] = X.BsmtQual.fillna("NA")
X["BsmtCond"] = X.BsmtCond.fillna("NA")
X["BsmtExposure"] = X.BsmtExposure.fillna("NA")
X.BsmtFinType2.replace({"NA":0, "Unf":1,"LwQ":2,"Rec":3,"BLQ":4,"ALQ":5, "GLQ":6},inplace=True)
X.BsmtFinType1.replace({"NA":0, "Unf":1,"LwQ":2,"Rec":3,"BLQ":4,"ALQ":5, "GLQ":6},inplace=True)
X.BsmtQual.replace({"NA":0, "Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5},inplace=True)
X.BsmtCond.replace({"NA":0, "Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5},inplace=True)
X.BsmtExposure.replace({"NA":0, "No":1,"Mn":2,"Av":3,"Gd":4},inplace=True)
#two rows with NaNs have no basement
X["BsmtFullBath"] = X.BsmtFullBath.fillna(0)
X["BsmtHalfBath"] = X.BsmtHalfBath.fillna(0)

#garage features
#157 rows has all garage features empty
#row 2127 misses GarageYrBlt and GarageFinish
#row 2577 has only type=Detchd
X["GarageType"] = X.GarageType.fillna("NA")
X["GarageFinish"] = X.GarageFinish.fillna("NA")
X["GarageQual"] = X.GarageQual.fillna("NA")
X["GarageCond"] = X.GarageCond.fillna("NA")
X["GarageCars"] = X.GarageCars.fillna(0)
X["GarageArea"] = X.GarageArea.fillna(0)
#assume that if we dont know the year it was built it is the same year than the house
X.GarageYrBlt.fillna(X.YearBuilt, inplace=True)
#highly opinionated scoring
X.GarageType.replace({"NA":0, "CarPort":1,"Detchd":2,"Basment":3,"BuiltIn":4,"Attchd":5,"2Types":6},inplace=True)
X.GarageFinish.replace({"NA":0, "Unf":1,"RFn":2,"Fin":3},inplace=True)
X.GarageQual.replace({"NA":0, "Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5},inplace=True)
X.GarageCond.replace({"NA":0, "Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5},inplace=True)
X.PavedDrive.replace({"N":1, "P":2,"Y":3},inplace=True)

#pool features
#PoolArea nothing to do
X["PoolQC"] = X.PoolQC.fillna("NA")
X.PoolQC.replace({"NA":0, "Fa":2,"TA":3,"Gd":4,"Ex":5},inplace=True)

#fence feature
X["Fence"] = X.Fence.fillna("NA")
X.Fence.replace({"NA":0, "MnWw":1,"GdWo":2,"MnPrv":3,"GdPrv":4},inplace=True)

#utilities features
X["Heating"] = lb_make.fit_transform(X["Heating"])
X.HeatingQC.replace({"NA":0, "Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5},inplace=True)
X.CentralAir.replace({"N":0, "Y":1},inplace=True)
#one row with NaN but All Public utilities
X["Electrical"] = X.Electrical.fillna("Mix")
X["Electrical"] = lb_make.fit_transform(X["Electrical"])
#2 NaNs but have electrical installation, all other disputable
X["Utilities"] = X.Utilities.fillna("ELO")
X.Utilities.replace({"ELO":1,"NoSeWa":2,"NoSewr":3,"AllPub":4},inplace=True)

X["MiscFeature"] = X.MiscFeature.fillna("NA")
X["MiscFeature"] = lb_make.fit_transform(X["MiscFeature"])
X["Functional"] = X.Functional.fillna("Typ")
X["Functional"] = lb_make.fit_transform(X["Functional"])
X["SaleType"] = X.SaleType.fillna("Oth")
X["SaleType"] = lb_make.fit_transform(X["SaleType"])
X["SaleCondition"] = lb_make.fit_transform(X["SaleCondition"])

#NEW FEATURES
#regular shape area
X['Regular'] = np.where(X['LotShape'] == 4, 1, 0)
#flat area
X['Flat'] = np.where(X['LandContour'] == 4, 1, 0)
#livable place
X['Livable'] = np.where(X.Utilities.isin([4,3]), 1, 0)
#overall value
X['OverallV']=X['OverallQual']*X['OverallCond']
#exterior value
X['ExterV']=X['ExterQual']*X['ExterCond']
#basement value
X['BsmtV']=X['BsmtQual']*X['BsmtCond']

TypeError: argument must be a string or number

In [ ]:
###### SPLIT DATA #####
#break it down again in train and test
x,x_predict = X.xs(0),X.xs(1)

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [ ]:
###### THIS IS THE MODEL SECTION ######
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(x_train, y_train)

# Use the forest's predict method on the validation data
y = rf.predict(x_val)

In [ ]:
#Calculate RMSE (used by the challenge for error)
print('RMSE:', round(np.sqrt(sum(np.square(np.log(y) - np.log(y_val.to_numpy())))/len(y)),5))
# Calculate the absolute errors
error = abs(y - y_val)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(error), 2))
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (error / y_val)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
#Feature importance
feature_list=X.columns.values
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('{:20} Importance: {}'.format(*pair)) for pair in feature_importances];

In [ ]:
###### PREDICTIONS!!! ######
y_predict = rf.predict(x_predict)

In [ ]:
# Save predictions to a file for submission
output = pd.DataFrame({'Id': Ids,
                       'SalePrice': y_predict})
output.to_csv('submission.csv', index=False)

#create a link to download the file    
from IPython.display import FileLink
FileLink(r'submission.csv')